### Simulation des rapports des visites bimestrielles et des interventions de maintenance annuelle des issues et des niches 

* Définition du calendier et des numéros d'OT (fermeturesJours.csv)
* Production des rapports (absences aléatoires)
* Upload des rapports sur le serveur

Cette simulation permet de tester les choix qui sont faits sur la forme des rapports.  
la simulation fournit un jeu de données pour développer les outils de traitements des données pour la visualisation des résultats.



In [22]:
#! pip install -r requirements.txt
import pandas as pd, numpy as np
import glob, re,json,io,os
from datetime import datetime, timedelta
from time import sleep
import plotly.graph_objects as go
pd.options.display.max_colwidth = 100
from numpy.random import random
from numpy.random import choice
rng = np.random.default_rng()
from google.cloud import bigquery,storage
project_id = 'tunnels-dirif'
clientB = bigquery.Client(project_id)
storageC=storage.Client(project_id)

### Programmation des dates de fermetures et simulation des 
Pour chaque fermeture, on définit aléatoirement 6 dates de visites bimestrielles (champ "jour").  
Les dates sont identifiées par le rang du jour dans l'année (dayofyear).   
On définit les OTs père (champ "OTP" : 456000+k*100+i  ). Les OTPs sont dans la plage 456000+456600  
On simule que des visites bimestrielles ne sont pas faites ou sont faites, mais reportée.  
Pour cela, on définit le champ 'faitFerm' avec mes valeurs (OK : fait à la date programmée, RP: fait mais reporté 21 jours plus tard & KO: non fait).    
Paramètres du calcul : ```tauxRéalisation & tauxReport```   

On crée la table des visites d'issues programmées `VisiteIssues`
On associe aux issues les dates de leurs fermetures programmées .    
On définit les OT fils (OTFs) des visites d'issues : ordreVB*300+indexIss+456600  
On simule que lors d'une visites bimestrielles certaines issues ne donnent pas lieu à la transmission d'un rapport.   
Pour cela, on définit le champ 'faitIss' avec mes valeurs (True / False).  
Paramètres du calcul : `tauxAbsenceIss`

On crée la table des rapport de visites d'issues transmis (faits) `VisiteIssuesFt` en sélectionnant avec les champs ```faitFerm & faitIss```    
On définit l'horodate de transmission du rapport par une valeur aléatoire comprise entre jour + 23h et jour + 27h (jour+1 +3h)   
Pour les visites reportées (``` faitIss = 'RP' ```) on calcule une nouvelle horodate 21 jours plus tard (on aurait pu faire varier le report mais on a simplifié)   
Enfin, on choisit aléatoirement un nom d'agent (il aurait été plus logique de choisir les agents au niveau de la fermeture mais c'est plus simple comme ça).



In [18]:

issues=pd.read_csv('https://raw.githubusercontent.com/ExploitIdF/Referentiel_Tunnels/refs/heads/main/issuesFermetures.csv')#[[  'Tatouage',   'Fermeture']]
# ['CodeEx', 'PC', 'Tatouage', 'triCode', 'Fermeture']
issues['OQ']=range(290)
fermetures=issues['Fermeture'].value_counts()

tauxRéalisation =0.9  # une fermeture à une date, dont report
tauxReport=0.1
tauxAbsenceIss=0.05 # pour les visites d'une fermeture, taux d'issues sans production d'un rapport
lenFer=len(fermetures )
if False:  # le process étant aléatoire, le refaire tourner modifie les résultats
  fermJours=[]
  for k in range(6):
    jfTmp=pd.Series([(5+k*8)*7+ int(random()*4)*7+int(random()*4) for i in range(lenFer)],index=fermetures.index,name='jour')
    otTmp=pd.Series([456000+k*100+i for i in range(lenFer)],index=fermetures.index,name='OTP')
    ordTmp=pd.Series( k,index=fermetures.index,name='ordreVB')
    fermJours.append(pd.concat([jfTmp,otTmp,ordTmp],axis=1))
  fermJours=pd.concat(fermJours).reset_index()
  fermJours['faitFerm']=[choice(a=['OK','RP','KO'],p=[tauxRéalisation- tauxReport,tauxReport,1-tauxRéalisation]) for i in range(lenFer*6)]
  fermJours.to_csv('fermJours.csv',index=False)

  VisiteIssues=issues.merge(fermJours,on='Fermeture', how='outer')
  VisiteIssues['OTF']=VisiteIssues['ordreVB']*300+VisiteIssues['OQ']+456600
  VisiteIssues['faitIss']=[random()>tauxAbsenceIss for i in range(len(VisiteIssues))]
  VisiteIssues[['Fermeture','ordreVB','faitFerm','faitIss', 'OTF','CodeEx', 'Tatouage', 'jour']].to_csv('VisiteIssues.csv',index=False)
  VisiteIssuesFt=VisiteIssues[(VisiteIssues['faitIss'])&(VisiteIssues['faitFerm']!='KO')].copy()
  VisiteIssuesFt['minute']=[int(random()*240) for i in range(len(VisiteIssuesFt))]
  VisiteIssuesFt['HoroDate']=pd.to_datetime("24/"+VisiteIssuesFt['jour'].astype(str)+" 23",format='%y/%j %H')+pd.to_timedelta(VisiteIssuesFt['minute']*60000000000) 
  VisiteIssuesFt.loc[VisiteIssuesFt['faitFerm']=='RP','HoroDate']=VisiteIssuesFt.loc[VisiteIssuesFt['faitFerm']=='RP','HoroDate']+pd.to_timedelta(21*24*60*60000000000)
  VisiteIssuesFt['agent']=[choice(['Karl','Karen', 'Kim','Kamel','Kun']) for k in range(len(VisiteIssuesFt))]
  VisiteIssuesFt[[ 'OTF','CodeEx', 'Tatouage','agent','jour', 'HoroDate']].to_csv('VisiteIssuesFt.csv',index=False)

In [19]:
# Lecture du tableau des points de contrôle initial. Ce format devrait changer ...
vbIa=pd.read_csv('../_static/controleVB_IA.csv',encoding='UTF-8').iloc[:,1:]
lstPC=vbIa['PointControle'].value_counts().reset_index()
lstPC['ordrePC']=lstPC['PointControle'].str.split('.').apply(lambda x: x[0]).astype(int).astype(str).str.zfill(2) 
lstPC=lstPC.sort_values('ordrePC').reset_index(drop=True) #.set_index('ordrePC',drop=True)
lstLst=[]
for i in range(len(lstPC)) :
  vbIi=vbIa[vbIa['PointControle']==lstPC.loc[i,'PointControle']]
  lstCom=[]
  for j in vbIi.index:
    lstCom.append(vbIi.loc[j,'Note'][0]+'_'+vbIi.loc[j,'ResulControle'])
  lstLst.append(sorted(lstCom, reverse=False))

lstNbrResul=[len(ll) for ll in lstLst]
lsP=[[round(.2/(i-1),3)]*(i-1) +[1-(i-1)*round(.2/(i-1),3)]    for i in lstNbrResul]
lstLstR=[['R'+str(i) for i in range(j)] for j in lstNbrResul ]

tabRC=[]
for i in range(len(lstPC)) :
  for j in range(lstNbrResul[i]) :
    tabRC.append(['P'+ lstPC.loc[i,'ordrePC'], lstPC.loc[i,'PointControle'],lstLstR[i][j],lstLst[i][j],lsP[i][j]  ] )
pd.DataFrame(tabRC,columns=['codePC','PC','codeRC', 'RC', 'Probab']).to_csv('controleVB_IS.csv',index=False)
# Dans le futur, on devrait entrer directement une table de ce type.


## Création des rapports
On lit la table des point&résultats de contrôles et leur probabilité.
On lit la liste des visites "faites".
On génère aléatoirement, pour chaque visite et chaque point de controle, un résultat de contrôle et un commentaire.

In [20]:
controleVB_IS=pd.read_csv('controleVB_IS.csv')
VisiteIssuesFt=pd.read_csv('VisiteIssuesFt.csv')  # visites d'issues faites
VisiteIssuesFt['HoroDate']=pd.to_datetime( VisiteIssuesFt['HoroDate'],format='%Y-%m-%d %H:%M:%S')
fermJours=pd.read_csv('fermJours.csv')
ComAls=[x for x in  ['Commentaire Aléatoire0 Très compliqué !','Commentaire Aléatoire1 Comment faire ?',
    'Commentaire Aléatoire2 Trop long à vous expliquer, on appelera le PCTT demain','Commentaire Aléatoire3 Trois choses à noter',
                     'Commentaire Aléatoire4 Sans commentaire']]
nbrVBF=len(VisiteIssuesFt)
codePCs=controleVB_IS['codePC'].unique()
nbrPC=len(codePCs)
RCs=[list(controleVB_IS[controleVB_IS['codePC']==codeP]['codeRC']  ) for codeP in codePCs ] # listes des résultats de contrôle par point de contrôle 
PRBs=[list(controleVB_IS[controleVB_IS['codePC']==codeP]['Probab'] ) for codeP in codePCs ] # listes des probabilité des résultats de contrôle par point de contrôle 
nbrRCs=[len(x) for x in RCs ]

lstRC=[]
for k in range(nbrVBF): 
   for n in range(nbrPC):
      lstRC.append([k,codePCs[n],choice(a=RCs[n],p=PRBs[n]),choice(ComAls) ])
lstRC=pd.DataFrame(lstRC,columns=['indRap', 'PC','RC','Com'])
lstRC.to_csv('lstRC.csv',index=False)

In [24]:
# Importation de la table des visites faites dans BigQuery (4 secondes)

client  = clientB
dataset  = client.dataset('rapports_visites')
table = dataset.table('VisiteIssuesFt')

job_config = bigquery.LoadJobConfig(
    schema=[ bigquery.SchemaField(cl, bigquery.enums.SqlTypeNames.STRING) for cl in VisiteIssuesFt.columns],
    write_disposition="WRITE_TRUNCATE",
       autodetect=False,
    source_format=bigquery.SourceFormat.CSV
)
job = client.load_table_from_dataframe( VisiteIssuesFt, table, job_config=job_config)  
job.result() 


LoadJob<project=tunnels-dirif, location=US, id=04a2900e-6c5e-4a47-8835-4de0333e8ced>

In [25]:
table = dataset.table('lstRC')

job_config = bigquery.LoadJobConfig(
    schema=[ bigquery.SchemaField(cl, bigquery.enums.SqlTypeNames.STRING) for cl in lstRC.columns],
    write_disposition="WRITE_TRUNCATE",
       autodetect=False,
    source_format=bigquery.SourceFormat.CSV
)
job = client.load_table_from_dataframe( lstRC, table, job_config=job_config)  
job.result() 

LoadJob<project=tunnels-dirif, location=US, id=a16893ca-eb09-4765-9ed4-954e10b89b45>

Les cellules qui suivent correspondent à une ancienne implémentation qui est caduque a vu de cde qui précède.
Conservé pour mémoire mais à reprendre entièrement

In [ ]:



def simuRapp(ferm,ord):
    vst=VisiteIssues[(VisiteIssues['Fermeture']==ferm)&(VisiteIssues['ordreVB']==ord)].reset_index(drop=True)  
    for k in range(len(vst)):
        if random()<.95:
            dateFichier= str(int((vst.loc[k,'HoroDate']-datetime(2024,1,1)).total_seconds()+1000*random()))
            flNm='24-3/ISVB-'+ str(vst.loc[k,'OT'])+'-'+dateFichier+ '.json'
            flSr='{"Tatouage":"'+str(VisiteIssues.loc[k,'Tatouage'])+'",'
            flSr=flSr+'"HoroDate":"'+str(VisiteIssues.loc[k,'HoroDate'])+'",'
            flSr=flSr+'"Agent":"'+choice(['Karl','Karen', 'Kim','Kamel','Kun']) +'"'
            for n in range():
                cn=controleVB_IS[controleVB_IS['codePC']== controleVB_IS['codePC'].unique()[n] ][['codeRC','Probab']]
                flSr=flSr+',"PC'+str(n)+'":"'+choice(a=list(cn['codeRC']),p=list(cn['Probab']))+'"'+' ,"CM'+str(n)+'":"'+choice(a=ComAls)+'"'            
            flSr=flSr+'}'
            with open(flNm,'bw') as fl:
                fl.write(flSr.encode('UTF8'))

# Création des rapports par application de la fonction avec une probabilité d'omission
repertoire ='24-3/'
if True :
    filenames = next(os.walk(repertoire), (None, None, []))[2]
    for fl in filenames:
        os.remove(repertoire + fl)

    for ferm in list(VisiteIssues['Fermeture'].unique()):
        for j in range(6):
            if random()<0.5:
                simuRapp(ferm,j)
filenames = next(os.walk(repertoire), (None, None, []))[2]



TypeError: 'int' object is not iterable

## Upload des rapports sur le serveur Google Storage
On vide le répertoire cible (Storage : issues-secours/rapports-visites/) et  on charge les fichiers avec une pause de 3 secondes 
pour tenir compte de l'import dans BQ par la fonction qui est déclenchée par le chargement.



In [ ]:
# On vide le répertoire cible (Storage : issues-secours/rapports-visites/) et on recharge les fichiers avec une pause de 3 seconde pour tenir compte de l'import dans BQ
bucket = storageC.get_bucket('issues-secours')
blobs = storageC.list_blobs(bucket)
for blob in blobs:
    if 'ISV' in blob.name:
        blob.delete()

for fl in filenames[:]:
    blob = bucket.blob('rapports-visites/'+fl)
    blob.upload_from_filename('24-3/'+ fl, if_generation_match= 0)
    sleep(3)


In [ ]:
# Liste des clés pour alimenter le schéma d'importation dans BQ
name = 'rapports-visites/' +filenames[5]
blob = bucket.blob(name)
fileContent= (blob.download_as_string(client=None).decode())
json.loads(fileContent).keys()

dict_keys(['Tatouage', 'HoroDate', 'Agent', 'PC0', 'CM0', 'PC1', 'CM1', 'PC2', 'CM2', 'PC3', 'CM3', 'PC4', 'CM4', 'PC5', 'CM5', 'PC6', 'CM6', 'PC7', 'CM7', 'PC8', 'CM8', 'PC9', 'CM9', 'PC10', 'CM10', 'PC11', 'CM11', 'PC12', 'CM12', 'PC13', 'CM13', 'PC14', 'CM14'])

In [ ]:
# Test de l'importation dans BQ pour la fonction logDepot
client  = bigquery.Client()
dataset  = client.dataset('rapports_visites')
table = dataset.table('LogDepot')

def format_schema(schema):
        formatted_schema = []
        for row in schema:
            formatted_schema.append(bigquery.SchemaField(row,'STRING', 'NULLABLE'))
        return formatted_schema
lst_schema_VBIS = ['Tatouage', 'HoroDate', 'Agent', 'PC0', 'CM0', 'PC1', 'CM1', 'PC2', 'CM2', 'PC3', 'CM3', 'PC4', 'CM4', 'PC5', 'CM5', 'PC6', 'CM6', 
                  'PC7', 'CM7', 'PC8', 'CM8', 'PC9', 'CM9', 'PC10', 'CM10', 'PC11', 'CM11', 'PC12', 'CM12', 'PC13', 'CM13', 'PC14', 'CM14']


job_config = bigquery.LoadJobConfig()
job_config.schema = format_schema(lst_schema_VBIS)
flJson=json.loads(fileContent)
stByt=','.join([flJson[k] for k in lst_schema_VBIS  ]).encode("utf-8")
job = client.load_table_from_file(io.BytesIO(stByt), table, job_config = job_config)